# SparkSQL

En mi caso tuve problemas conectando los servidores (EN LINUX)!
por eso te recomiento ver los sigueintes tutoriales por si tienes problemas al igual que yo!

[video](https://youtu.be/H7LiT-MkriU?si=aIDdESCGhiUIU0CK)

[pagina](https://medium.com/@abdulhaque.dev/installing-and-configuring-spark-with-mongodb-on-ubuntu-b74a06f9c965) (Para descargar los dos archivos .JAR y meterla dentro de los archivos JARS de la carpeta spark)

Ademas de esto! Te tecomiendo ejecutar mongodb-compass, si tu instalacion fue exitosa abrelo desde la consola `mongodb-compass`

Ademas de esto, deja dos consolas abiertas con spark activo: `spark-shell`
y mongod activo: `sudomongod`

In [1]:
# Import necessary libraries 
from pyspark.sql import SparkSession

# State the input and output directories of your Database 
input_uri = "mongodb://localhost:27017/test.ciudades"
output_uri = "mongodb://localhost:27017/test.ciudades"

# Make a spark session to use pysaprk and also configure spark to be configured with MongoDB as follows
spark = SparkSession.builder \
    .appName("myProject") \
    .config("spark.mongodb.input.uri", input_uri) \
    .config("spark.mongodb.output.uri", output_uri) \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.2") \
    .getOrCreate()
    
spark

24/05/19 19:31:53 WARN Utils: Your hostname, miguel-9051-900-0098 resolves to a loopback address: 127.0.1.1; using 192.168.1.58 instead (on interface enp6s0)
24/05/19 19:31:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/miguel/Descargas/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/miguel/.ivy2/cache
The jars for the packages stored in: /home/miguel/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d9b69ff8-6fa6-404f-a3af-404167136969;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.2 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 182ms :: artifacts dl 7ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts 

In [2]:
sc = spark.sparkContext

## Creación de DataFrames

In [3]:
l = [('a',3.14), ('b', 9.4), ('a',2.7)]
headers = ['id','value']
df = spark.createDataFrame(l, headers)
n = 3 #
df.show(n) # <-- Nos mostrara los primero n elementos del dataframe (por defecto son 20)

df.printSchema() # <-- Spark infiere un esquema de datos, que podemos verlo asi

+---+-----+
| id|value|
+---+-----+
|  a| 3.14|
|  b|  9.4|
|  a|  2.7|
+---+-----+

root
 |-- id: string (nullable = true)
 |-- value: double (nullable = true)



In [4]:
l = [('a',3.14), ('b',True)]
headers = ['id','value']
## df = spark.createDataFrame(l, headers) 
## Esta invocación lanzaría una excepción 
## TypeError: field value: Can not merge type <class 'pyspark.sql.types.DoubleType'> and <class 'pyspark.sql.types.BooleanType'>

si no le pasamos los parametros del header el los rellenara automaticamente con valores consecutivos _1, _2, _3

In [5]:
l = [('a',1), ('b', 2), ('a',3)]
df = spark.createDataFrame(l)
df.show(n)

df.printSchema() # <-- Spark sigue infiriendo el esquema de datos

+---+---+
| _1| _2|
+---+---+
|  a|  1|
|  b|  2|
|  a|  3|
+---+---+

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)



In [6]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType

l = [('a',3.14), ('b', 9.4), (None, 5.7)]
# VAMOS A ASIGNARUN ESQUEMA EN UN DATAFRAME, PARA ESO DEBEMOS CREAR UN OBJETO DE TIPO ** StrucType **
# Que contenga tantos objetos StructField como columnas vaya a temner nuestro DataFrame
schema = StructType([
  #           ESTE ACEPTA 3 PARAMATROS
  #           Columna, TIpo de dato, admite valores booleanos?
  StructField('id',   StringType(), True),
  StructField('value', FloatType(), False)
])
df = spark.createDataFrame(l, schema)
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- value: float (nullable = false)



In [7]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType

l = [('a',3.14), ('b', 9.4), ('a', True)]
schema = StructType([
  StructField('id',   StringType(), True),
  StructField('value', FloatType(), False)
])
## df = spark.createDataFrame(l, schema) ## Lanza excepción
## TypeError: field value: FloatType can not accept object True in type <class 'bool'>

# Error porque solo peude contener valoresde tipo flotante

spak context!

In [8]:
r = sc.parallelize([('a',3.14), ('b', 9.4), ('a', 2.7)])
df = spark.createDataFrame(r, ['id','value'])
df.show()
df.printSchema()

+---+-----+
| id|value|
+---+-----+
|  a| 3.14|
|  b|  9.4|
|  a|  2.7|
+---+-----+

root
 |-- id: string (nullable = true)
 |-- value: double (nullable = true)



##  Creacion de un DataFrame a partir de un RDD

In [9]:
# Creación de un DataFrame a partir de un RDD
import csv

raw = (
  sc.textFile("../../data/Cap7/titanic.csv") # Leemos el fichero de texto
    .map(lambda s: list(csv.reader([s]))[0]) # Dividimos el CSV en listas
    .filter(lambda l: l[0] != 'PassengerId') # Eliminamos la cabecera, entonces por defecto quedan numeros...
) # <---- ESTE ES EL RDD

df = spark.createDataFrame(raw)
df.show(3)
df.printSchema()
headers = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
df = spark.createDataFrame(raw, headers)
df.show(3)
df.printSchema()

+---+---+---+--------------------+------+---+---+---+----------------+-------+---+---+
| _1| _2| _3|                  _4|    _5| _6| _7| _8|              _9|    _10|_11|_12|
+---+---+---+--------------------+------+---+---+---+----------------+-------+---+---+
|  1|  0|  3|Braund, Mr. Owen ...|  male| 22|  1|  0|       A/5 21171|   7.25|   |  S|
|  2|  1|  1|Cumings, Mrs. Joh...|female| 38|  1|  0|        PC 17599|71.2833|C85|  C|
|  3|  1|  3|Heikkinen, Miss. ...|female| 26|  0|  0|STON/O2. 3101282|  7.925|   |  S|
+---+---+---+--------------------+------+---+---+---+----------------+-------+---+---+
only showing top 3 rows

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: string (nullable = true)
 |-- _7: string (nullable = true)
 |-- _8: string (nullable = true)
 |-- _9: string (nullable = true)
 |-- _10: string (nullable = true)
 |-- _11: string (nu

## Lectura y escritura de DataFrames desde ficheros

In [10]:
df = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



Aunque no le pasemos parametros en json printSchema tambein peude recibr parametros como csv!
Aunque no hay problema si queda vacia, ya que sprk intentaq predecirlos

In [11]:
df = spark.read.json('../../data/Cap7/tweets.json') # aca no le pasamos parametros, y que spark por defecto los interpreta!
df.printSchema()
df.show()

root
 |-- RT_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- verified: boolean (nullable = true)

+--------+--------------+----------------+
|RT_count|          text|            user|
+--------+--------------+----------------+
|       2|   #Tengosueño| {3, Pepe, true}|
|      45|  #VivaElLunes|{15, Ana, false}|
|     100|¡Gol de Señor!|  {2, Eva, true}|
+--------+--------------+----------------+



In [12]:
## Lee todos los ficheros, y cada línea es una fila del DataFrame.
## El DataFrame tiene una única columna 'value' de tipo string
df = spark.read.text('../../data/Cap7/*.txt')
df.printSchema()
df.show()

root
 |-- value: string (nullable = true)

+--------------------+
|               value|
+--------------------+
|En el anterior ap...|
|y que se crean me...|
|antes de poder co...|
|un RDD (o varios)...|
|Estos RDDs se cre...|
|o a partir de fic...|
|creará un RDD y s...|
|diferentes proces...|
|En el anterior ap...|
|y que se crean me...|
|antes de poder co...|
|un RDD (o varios)...|
|Estos RDDs se cre...|
|o a partir de fic...|
|creará un RDD y s...|
|diferentes proces...|
+--------------------+



In [13]:
## Lee todos los ficheros, y cada ficheros es una fila del DataFrame
## El DataFrame tiene una única columna 'value' de tipo string
df = spark.read.text('../../data/Cap7/*.txt', wholetext=True)
df.printSchema()
df.show()

root
 |-- value: string (nullable = true)

+--------------------+
|               value|
+--------------------+
|En el anterior ap...|
|En el anterior ap...|
+--------------------+



## Almacenaiento de Dataframes

In [14]:
# Escritura de un DataFrame de 50 elementos en la carpeta /tmp/csv
l = [('a',3.14)] * 50
df = spark.createDataFrame(l, ['id','value'])
df.printSchema()
df.write.csv('/tmp/csv', header=True, mode='overwrite') # recordemos quese guardara al inicio de nuestro disco!

root
 |-- id: string (nullable = true)
 |-- value: double (nullable = true)



si queiremos cambiar el parametro de separacion podemosa usar `sep` y para incluir las cabeceras en cada fichero utilizariamos `header=True`

Otro paraetro interesante es el modo de escritura `mode`, que nos permitira congfgurar que hacer si la carpeta ya existe. Este acepta 4 valores, `append` que agregara los datos a los ficheros ya existentes, `overwrite` los sobreescribiremos completramente, `ignore` evitaremos cualquier escritura si existen ya ficheros, y `error` lanzaremos una excepecion si la carpeta ya existia anteriormente.

El metodo spark.read nos permite pasar como parametro una carpeta, por lo cual no sera un problema quese escriban varios ficheros al tiempo

### CSV

In [15]:
# Lectura del DataFrame anterior desde la carpeta /tmp/csv
df = spark.read.csv('/tmp/csv', header=True, inferSchema=True) # todos los elmentos de esa carpeta
df.printSchema()
df.show(4)
df.count()

root
 |-- id: string (nullable = true)
 |-- value: double (nullable = true)

+---+-----+
| id|value|
+---+-----+
|  a| 3.14|
|  a| 3.14|
|  a| 3.14|
|  a| 3.14|
+---+-----+
only showing top 4 rows



50

### JSON

In [16]:
l = [('a',3.14)] * 50
df = spark.createDataFrame(l, ['id','value'])
df.printSchema()
print(df.rdd.getNumPartitions())
df.write.json('/tmp/json', mode='overwrite')

root
 |-- id: string (nullable = true)
 |-- value: double (nullable = true)

12


In [17]:
# Lectura del DataFrame anterior desde la carpeta /tmp/json
df = spark.read.json('/tmp/json') # Aqui no debemos
df.printSchema()
df.show(4)
df.count()

root
 |-- id: string (nullable = true)
 |-- value: double (nullable = true)

+---+-----+
| id|value|
+---+-----+
|  a| 3.14|
|  a| 3.14|
|  a| 3.14|
|  a| 3.14|
+---+-----+
only showing top 4 rows



50

Crear un Dataframe de Pandas a partir de un Dataframe de spark

In [18]:
#pip install distutils
# Volcar un DataFrame a Pandas
l = [('a',3.14)] * 12
df = spark.createDataFrame(l, ['id','value'])
df_pandas = df.toPandas()

## DataFrames y MongoDB

Para poder cargar y salvar DataFrames en MongoDB es necesario lanzar ```pyspark``` configurando el conector de Mongo-Spark. Para ello es necesario pasar el siguiente parámetro a la hora de invocar a ```pyspark```:

Iiciar mongo:
```shell
sudo systemctl start mongod
```

verifica que esta activo:

```shell
sudo systemctl status mongod
```

empieza a usarlo:

```shell
mongosh
```

si acabas peude finaizar con:

```shell
sudo systemctl stop mongod
```


mongod db documentacion: [aqui](https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-ubuntu/)


```
pyspark --packages org.mongodb.spark:mongo-spark-connector_2.11:2.2.2 --master local[*]
```
        
El nombre del conector corresponde a _coordenadas_ Maven:
 * **2.11** porque el conector utiliza la versión 2.11 de Scala
 * **2.2.2** es la versión concreta del conector. La versión 2.2.2 soporta MongoDB 2.2.x y 2.3.x
 
El parámetro ```--master``` es el usual. En este caso lanzamos ```pyspark``` en modo local y utilizando tantos procesos _workers_ como núcleos tenga nuestra CPU.

Se puede encontrar más información en:
 * https://docs.mongodb.com/spark-connector/master/
 * https://docs.mongodb.com/spark-connector/master/python-api/
 
**Supondremos que existe un servidor MongoDB ejecutándose en local (IP 127.0.0.1) y escuchando en el puerto por defecto (27017).**

In [19]:
# Volcar un DataFrame a una colección MongoDB

ciudades = spark.createDataFrame([("Madrid",  3182981), ("Barcelona", 1620809), ("Valencia",787808),
                                  ("Sevilla", 689434), ("Zaragoza", 664938), ("Málaga",569002), 
                                  ("Murcia",443243), ("Palma",406492) ], ["nombre", "habitantes"])
ciudades.show()
ciudades.printSchema()

ciudades.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").save()

# *Importante*: La colección test.ciudades no debe existir. Si lo que queremos es añadir a una colección ya existente, debemos usar mode("append"):

+---------+----------+
|   nombre|habitantes|
+---------+----------+
|   Madrid|   3182981|
|Barcelona|   1620809|
| Valencia|    787808|
|  Sevilla|    689434|
| Zaragoza|    664938|
|   Málaga|    569002|
|   Murcia|    443243|
|    Palma|    406492|
+---------+----------+

root
 |-- nombre: string (nullable = true)
 |-- habitantes: long (nullable = true)



In [20]:
# Cargar un DataFrame desde una colección MongoDB
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.ciudades").load()
df.show()
df.printSchema()

+--------------------+----------+---------+
|                 _id|habitantes|   nombre|
+--------------------+----------+---------+
|{664230d2b7e3841e...|   1620809|Barcelona|
|{664230d2b7e3841e...|   3182981|   Madrid|
|{664230d2b7e3841e...|    406492|    Palma|
|{664230d2b7e3841e...|    664938| Zaragoza|
|{664230d2b7e3841e...|    443243|   Murcia|
|{664230d2b7e3841e...|    689434|  Sevilla|
|{664230d2b7e3841e...|    569002|   Málaga|
|{664230d2b7e3841e...|    787808| Valencia|
|{66423252b7e3841e...|   1620809|Barcelona|
|{66423252b7e3841e...|    406492|    Palma|
|{66423252b7e3841e...|    787808| Valencia|
|{66423252b7e3841e...|    664938| Zaragoza|
|{66423252b7e3841e...|    443243|   Murcia|
|{66423252b7e3841e...|    569002|   Málaga|
|{66423252b7e3841e...|    689434|  Sevilla|
|{66423252b7e3841e...|   3182981|   Madrid|
|{664296b7e0e1be44...|    664938| Zaragoza|
|{664296b7e0e1be44...|    443243|   Murcia|
|{664296b7e0e1be44...|   1620809|Barcelona|
|{664296b7e0e1be44...|    406492

Vams a pasar todos las ciudades con mas de 500000 habitantes!

Esto lo hacemos gracias a que usamos las tuberias (pipelines) que usamos con mongo

In [21]:
pipeline = "{'$match': {'habitantes': {$gt:500000}}}"
masde500mil = (spark.read.format("com.mongodb.spark.sql.DefaultSource")
                        .option("uri","mongodb://127.0.0.1/test.ciudades")
                        .option("pipeline", pipeline).load()
              )
masde500mil.show()

+--------------------+----------+---------+
|                 _id|habitantes|   nombre|
+--------------------+----------+---------+
|{664230d2b7e3841e...|   1620809|Barcelona|
|{664230d2b7e3841e...|   3182981|   Madrid|
|{664230d2b7e3841e...|    664938| Zaragoza|
|{664230d2b7e3841e...|    689434|  Sevilla|
|{664230d2b7e3841e...|    569002|   Málaga|
|{664230d2b7e3841e...|    787808| Valencia|
|{66423252b7e3841e...|   1620809|Barcelona|
|{66423252b7e3841e...|    787808| Valencia|
|{66423252b7e3841e...|    664938| Zaragoza|
|{66423252b7e3841e...|    569002|   Málaga|
|{66423252b7e3841e...|    689434|  Sevilla|
|{66423252b7e3841e...|   3182981|   Madrid|
|{664296b7e0e1be44...|    664938| Zaragoza|
|{664296b7e0e1be44...|   1620809|Barcelona|
|{664296b7e0e1be44...|    787808| Valencia|
|{664296b7e0e1be44...|    569002|   Málaga|
|{664296b7e0e1be44...|    689434|  Sevilla|
|{664296b7e0e1be44...|   3182981|   Madrid|
|{66429c19e0e1be44...|    664938| Zaragoza|
|{66429c19e0e1be44...|    569002

In [22]:
# Cargar un DataFrame desde una colección MongoDB heterogénea (distintos documentos tienen atributos diferentes)
from pymongo import MongoClient #pip install pymongo

# Borramos la colección test.usuarios e insertamos dos documentos heterogéneos
client = MongoClient('127.0.0.1') # local host
col = client['test']['usuarios']
col.drop()
col.insert_many([{'nombre':'ana', 'edad':33}, {'nombre':'pedro','altura':150}]) # insertamos dos nuevs elementos (insert_many)

# Cargamos un DataFrame a partir de la colección test.usuarios
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.usuarios").load()# en la tabla test de la coleccion usuarios se guardaran los datos
df.show()
df.printSchema()

+--------------------+------+----+------+
|                 _id|altura|edad|nombre|
+--------------------+------+----+------+
|{664a9a08e67c9aa9...|  NULL|  33|   ana|
|{664a9a08e67c9aa9...|   150|NULL| pedro|
+--------------------+------+----+------+

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- altura: integer (nullable = true)
 |-- edad: integer (nullable = true)
 |-- nombre: string (nullable = true)



## Inspección de DataFrames

In [23]:
l = [('a',3.14), ('b',2.0), ('c',4.5)]
df = spark.createDataFrame(l, ['id','value'])
print(df.count()) # muestra las filas del dataframe

print(df.show())

3
+---+-----+
| id|value|
+---+-----+
|  a| 3.14|
|  b|  2.0|
|  c|  4.5|
+---+-----+

None


#### Metodo describe()

In [24]:
# 
l = [('a',3.14)] * 12
df = spark.createDataFrame(l, ['id','value'])
df.describe().show()
type(df.printSchema())

+-------+----+-----+
|summary|  id|value|
+-------+----+-----+
|  count|  12|   12|
|   mean|NULL| 3.14|
| stddev|NULL|  0.0|
|    min|   a| 3.14|
|    max|   a| 3.14|
+-------+----+-----+

root
 |-- id: string (nullable = true)
 |-- value: double (nullable = true)



NoneType

Vamos a analizar los datos del titanic con el metodo `describe()`

In [25]:
df = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
#### EL METODO DESCRIBE ACEPTA VARIOS PARAMETROS ####
df.describe(['Age','Fare','Sex','Cabin']).show()

24/05/19 19:32:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-----------------+------+-----+
|summary|               Age|             Fare|   Sex|Cabin|
+-------+------------------+-----------------+------+-----+
|  count|               714|              891|   891|  204|
|   mean| 29.69911764705882| 32.2042079685746|  NULL| NULL|
| stddev|14.526497332334035|49.69342859718089|  NULL| NULL|
|    min|              0.42|              0.0|female|  A10|
|    max|              80.0|         512.3292|  male|    T|
+-------+------------------+-----------------+------+-----+



## Filtrado de DataFrames

#### Eliminación de columnas con drop()

In [26]:
titanic = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
print(titanic.columns, " = Columnas seleccionadas")
df2 = titanic.drop('PassengerId','Name','Cabin')
print(df2.columns, " = eliminacion de 3 columnas")
df3 = titanic.drop(titanic.PassengerId)
print(df3.columns, " = eliminacion de la columna 'PassegerId'")
df4 = titanic.drop(titanic['PassengerId'])
print(df4.columns, " = eliminacion de la columna 'PassegerId'")

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']  = Columnas seleccionadas
['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']  = eliminacion de 3 columnas
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']  = eliminacion de la columna 'PassegerId'
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']  = eliminacion de la columna 'PassegerId'


#### Selección de columnas con select()

In [27]:
titanic = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
print(titanic.columns)
df2 = titanic.select('Survived', 'Pclass', 'Age') # seleccionamos estas columnas y eliminamos las otras :)
print(df2.columns)

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
['Survived', 'Pclass', 'Age']


#### seleccion de filas con dropDuplicates()

In [28]:
#este metodo nos permite eiminar los duplicados, podemos dejarlo vacio y tomara en cuent todas las columnas, aunque podemos pasarle parametros utiles
titanic = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
print(titanic.count())
df = titanic.dropDuplicates()
print(df.count())
#no se elimino ningun elemento porque todos son unicos!

891
891


en este caso dropDUplicates() eliminara todos los valores menos los dos unicosvalores (Female Y Male...) 

Es decir, quedaran unicamente dos valores...

In [29]:
df = titanic.dropDuplicates(['Sex'])
print(df.count())

2


#### Descarte por dropna

Dispone de un prametro `how`, que indica que se debn eliminar las filas que tengan todos sus valores nulos (valor 'all') o si unicamente es necesario que contenga algun valo vacio para proceder a su eliminacion (valor 'any')

Tambien prmite definir cual es el minimo de numero de vaores no vacios que debe tener un fila para conservarla en el DataFrame (parametro thresh), este paramaetro invalidara el valor pasado por `how`

Tambien nospermite pasaruna list de nombres de columnas sobre las que buscar valores vacios en su parametro `subset`


Por defecto `dropna` eliminara aquellas filas que tengan un valor vacio en alguna de sus clumnas `how='any'`

In [30]:
titanic = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
print(titanic.count())
df = titanic.dropna() # elimina todos los que tegan al menos un valor vacio!
print(df.count()) #quedan unicamente 183 elementos
df = titanic.drop('Cabin').dropna() # elimina la columna Cabin, para luego eiminar los objetos vacios (no se elimnaran tatos)
print(df.count())

891
183
712


#### Operacion Filter

In [31]:
titanic = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
df = titanic.filter( 'Survived = 1') #sintaxis sql
print(df.count())
df = titanic.filter( df.Survived == 1) # esto hara que nos pase todos los sobrevivientes!
print(df.count())

342
342


las condiciones se pueden combinar

In [32]:
df = titanic.filter( 'Survived = 1 AND Sex = "female" AND Age > 20') # las mujeres de mas de 20 a;os!
print(df.count())
df = titanic.filter( (df.Survived == 1) & (df['Sex'] == 'female') & (df.Age > 20))
print(df.count())

144
144


In [33]:
# Filas cuyo camarote tiene 3 caracteres, el primero es una A o una B mayúsculas y el último un 2
# Usa una expresión regular
df = titanic.filter('Cabin RLIKE "^[AB].2$"')
print(df.count())
df = titanic.filter(df.Cabin.rlike('^[AB].2$'))
print(df.count())

4
4


## Combinación de DataFrames

Para la combinacion de columnas los Dataframes deben ser identicos en su numeros de filas y en sus tipos

Para unir dos Dataframes podemos usar el metodo `union`:

Union no elimina duplicados... es decir que si en alguno de los dos dataframes los elementos se repiten estaran duplicados al unirlos, pero igualmente podemos eliminarlos con `dropduplicates` como vimos anteriormente.

In [34]:
df1 = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
df2 = spark.createDataFrame([(3,'marta'),(1,'ana')],['id','nombre'])
df = df1.union(df2)
df.show()

+---+------+
| id|nombre|
+---+------+
|  1|   ana|
|  2|  jose|
|  3| marta|
|  1|   ana|
+---+------+



In [35]:
df1 = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
df1.printSchema()
df2 = spark.createDataFrame([(3.0,'marta'),(1.0,'ana')],['id','nombre'])
df2.printSchema()
df = df1.union(df2) ## No hay problema: long y double son compatibles => double
df.show()
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- nombre: string (nullable = true)

root
 |-- id: double (nullable = true)
 |-- nombre: string (nullable = true)

+---+------+
| id|nombre|
+---+------+
|1.0|   ana|
|2.0|  jose|
|3.0| marta|
|1.0|   ana|
+---+------+

root
 |-- id: double (nullable = true)
 |-- nombre: string (nullable = true)



In [36]:
df1 = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
df1.printSchema()
df2 = spark.createDataFrame([('3','marta'),('1','ana')],['id','nombre'])
df2.printSchema()
df = df1.union(df2) ## No hay problema: long y string son compatibles => string
df.show()
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- nombre: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- nombre: string (nullable = true)

+---+------+
| id|nombre|
+---+------+
|  1|   ana|
|  2|  jose|
|  3| marta|
|  1|   ana|
+---+------+

root
 |-- id: string (nullable = true)
 |-- nombre: string (nullable = true)



In [37]:
df1 = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
df1.printSchema()
df2 = spark.createDataFrame([(True,'marta'),(False,'ana')],['id','nombre'])
df2.printSchema()
## df = df1.union(df2) ## Excepcion Por tipo de datos
##AnalysisException: "Union can only be performed on tables with the compatible column types. boolean <> bigint at the first column of the second table

root
 |-- id: long (nullable = true)
 |-- nombre: string (nullable = true)

root
 |-- id: boolean (nullable = true)
 |-- nombre: string (nullable = true)



In [38]:
df1 = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
df1.printSchema()
df2 = spark.createDataFrame([(3,'marta',33),(4,'señor',44)],['id','nombre','edad'])
df2.printSchema()
## df = df1.union(df2)  ## Excepción Por tipo de datos
## AnalysisException: "Union can only be performed on tables with the same number of columns, but the first table has 2 columns and the second table has 3 columns

root
 |-- id: long (nullable = true)
 |-- nombre: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- edad: long (nullable = true)



In [39]:
df1 = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
df2 = spark.createDataFrame([(3,'marta'),(1,'ana')],['id','nombre'])
df = df1.intersect(df2) # 
df.show()

+---+------+
| id|nombre|
+---+------+
|  1|   ana|
+---+------+



In [40]:
df1 = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
df2 = spark.createDataFrame([(3,'marta'),(1,'ana')],['id','nombre'])
df = df1.subtract(df2)
df.show()

+---+------+
| id|nombre|
+---+------+
|  2|  jose|
+---+------+



### Metodo join
combina datos de dos DataFrames fusionado las filas que tienen los mismo valroes en coertas columnas:

In [41]:
## inner join de dos DataFrames sobre la columna 'id'
users = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
users.show()
age = spark.createDataFrame([(1,36),(2,30)],['id','edad'])
age.show()
df = users.join(age,'id')
df.show()

+---+------+
| id|nombre|
+---+------+
|  1|   ana|
|  2|  jose|
+---+------+

+---+----+
| id|edad|
+---+----+
|  1|  36|
|  2|  30|
+---+----+

+---+------+----+
| id|nombre|edad|
+---+------+----+
|  1|   ana|  36|
|  2|  jose|  30|
+---+------+----+



Esto funcionara siempre yu cuando los tipos de datos sean compatibles entre si

En este ejemplo se guardara el id como texto y como entero:

#### Inner join donde la columna 'id' contiene enteros y cadenas de texto

In [42]:
users = spark.createDataFrame([('1','ana'),('2','jose')],['id','nombre'])
users.printSchema()
age = spark.createDataFrame([(1,36),(2,30)],['id','edad'])
age.printSchema()
df = users.join(age,'id') ## No hay problema: string y long son compatibles => string
df.printSchema()
df.show()

root
 |-- id: string (nullable = true)
 |-- nombre: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- edad: long (nullable = true)

root
 |-- id: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- edad: long (nullable = true)

+---+------+----+
| id|nombre|edad|
+---+------+----+
|  1|   ana|  36|
|  2|  jose|  30|
+---+------+----+



#### inner join considerando igualdad de varias columnas a la vez

In [43]:
users = spark.createDataFrame([(1,'ana','golf'),(2,'jose','polo',)],['id','nombre','deporte'])
users.show()
age = spark.createDataFrame([(1,'eva',33),(2,'jose',30)],['id','nombre','edad'])
age.show()
df = users.join(age,["id", "nombre"])
df.show()

+---+------+-------+
| id|nombre|deporte|
+---+------+-------+
|  1|   ana|   golf|
|  2|  jose|   polo|
+---+------+-------+

+---+------+----+
| id|nombre|edad|
+---+------+----+
|  1|   eva|  33|
|  2|  jose|  30|
+---+------+----+

+---+------+-------+----+
| id|nombre|deporte|edad|
+---+------+-------+----+
|  2|  jose|   polo|  30|
+---+------+-------+----+



#### inner join usando una expresión de igualdad entre 2 columnas

In [44]:
users = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
users.show()
age = spark.createDataFrame([(1,36),(2,30)],['ident','edad'])
age.show()
df = users.join(age,users.id == age.ident)
df.show()

+---+------+
| id|nombre|
+---+------+
|  1|   ana|
|  2|  jose|
+---+------+

+-----+----+
|ident|edad|
+-----+----+
|    1|  36|
|    2|  30|
+-----+----+

+---+------+-----+----+
| id|nombre|ident|edad|
+---+------+-----+----+
|  1|   ana|    1|  36|
|  2|  jose|    2|  30|
+---+------+-----+----+



#### inner join usando una expresión de igualdad entre 4 columnas

In [45]:
users = spark.createDataFrame([(1,'ana','golf'),(2,'jose','polo',)],['id','name','sport'])
users.show()
age = spark.createDataFrame([(1,'eva',33),(2,'jose',30)],['ident','nombre','edad'])
age.show()
df = users.join(age,(users.id == age.ident) & (users.name == age.nombre))
df.show()

+---+----+-----+
| id|name|sport|
+---+----+-----+
|  1| ana| golf|
|  2|jose| polo|
+---+----+-----+

+-----+------+----+
|ident|nombre|edad|
+-----+------+----+
|    1|   eva|  33|
|    2|  jose|  30|
+-----+------+----+

+---+----+-----+-----+------+----+
| id|name|sport|ident|nombre|edad|
+---+----+-----+-----+------+----+
|  2|jose| polo|    2|  jose|  30|
+---+----+-----+-----+------+----+



#### inner join usando una expresión compleja entre 2 columnas
En este ejemplo vamos a tomar el segundfo valor si el primero es vacio, o tomamos el primero si el primero no es vacio!

In [46]:
users = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
users.show()
age = spark.createDataFrame([(None,1,33),(2,5,30)],['id1','id2','edad'])
age.show()
cond = ((age.id1.isNotNull() & (users.id == age.id1)) | 
       (age.id1.isNull() & (users.id == age.id2)))
df = users.join(age,cond)
df.show()

+---+------+
| id|nombre|
+---+------+
|  1|   ana|
|  2|  jose|
+---+------+

+----+---+----+
| id1|id2|edad|
+----+---+----+
|NULL|  1|  33|
|   2|  5|  30|
+----+---+----+



+---+------+----+---+----+
| id|nombre| id1|id2|edad|
+---+------+----+---+----+
|  1|   ana|NULL|  1|  33|
|  2|  jose|   2|  5|  30|
+---+------+----+---+----+



In [47]:
## left outer join de users y age
users = spark.createDataFrame([(1,'ana'),(2,'jose'),(3,'eva')],['id','nombre'])
users.show()
age = spark.createDataFrame([(1,36),(2,30)],['id','edad'])
age.show()
df = users.join(age,'id','left_outer')
df.show()

+---+------+
| id|nombre|
+---+------+
|  1|   ana|
|  2|  jose|
|  3|   eva|
+---+------+

+---+----+
| id|edad|
+---+----+
|  1|  36|
|  2|  30|
+---+----+

+---+------+----+
| id|nombre|edad|
+---+------+----+
|  1|   ana|  36|
|  2|  jose|  30|
|  3|   eva|NULL|
+---+------+----+



## Transformación de DataFrames

In [48]:
def load_titanic():
    return spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True).drop('Cabin').dropna()

#### Operaciones aritméticas entre columnas

EN este caso vamos a dejar unicamente 3 columnas:

In [49]:

titanic = load_titanic()
titanic.selectExpr("Survived","SibSp + Parch As Family", "Age * 12 AS Age").show(3)
#                  \__-> COLUMNA 1  \__-> Al sumar las dos sale una  \__--> Al operar Age * 12 creamos una nueva tabla!

+--------+------+-----+
|Survived|Family|  Age|
+--------+------+-----+
|       0|     1|264.0|
|       1|     1|456.0|
|       1|     0|312.0|
+--------+------+-----+
only showing top 3 rows



In [50]:
# Funciones definidas por el usuario (UDF) para transformar columnas
titanic = load_titanic()

def sex_to_num(s):
    ret = None
    if s == 'female':
        ret = 0
    elif s == 'male':
        ret = 1
    return ret

from pyspark.sql.types import IntegerType
spark.udf.register("sex_to_num", sex_to_num, IntegerType())

titanic.selectExpr("Sex", "sex_to_num(Sex) AS Sex_num").show(3)

+------+-------+
|   Sex|Sex_num|
+------+-------+
|  male|      1|
|female|      0|
|female|      0|
+------+-------+
only showing top 3 rows



In [51]:
# Funciones definidas por el usuario (UDF) para transformar columnas
titanic = load_titanic()

spark.udf.register("max_int", max, IntegerType())
    
titanic.selectExpr("SibSp", "Parch", "max_int(SibSp, Parch) AS Max_Family").show(3)

+-----+-----+----------+
|SibSp|Parch|Max_Family|
+-----+-----+----------+
|    1|    0|         1|
|    1|    0|         1|
|    0|    0|         0|
+-----+-----+----------+
only showing top 3 rows



In [52]:
# Funciones definidas por el usuario (UDF) para transformar columnas
titanic = load_titanic()

def scale(n,minv,maxv):
    return (n - minv) / (maxv - minv)

# Se puede obtener los valores minimos y maximos de cada columna a través del DataFrame generado por describe()
summary = titanic.describe().toPandas()
min_age = float(summary.loc[3,'Age'])
max_age = float(summary.loc[4,'Age'])

from pyspark.sql.types import DoubleType
spark.udf.register("scale_Age", lambda x: scale(x, min_age, max_age), DoubleType())

titanic.selectExpr("Age", "scale_Age(Age) AS Scaled_Age").show(3)

+----+-------------------+
| Age|         Scaled_Age|
+----+-------------------+
|22.0| 0.2711736617240513|
|38.0| 0.4722292033174164|
|26.0|0.32143754712239253|
+----+-------------------+
only showing top 3 rows



In [53]:
# Transformación de columnas usando expresiones entre columnas
titanic = load_titanic()
titanic.select(titanic.Survived,(titanic.SibSp + titanic.Parch).alias("Family"), (titanic.Age * 12).alias("Age")).show(3)

+--------+------+-----+
|Survived|Family|  Age|
+--------+------+-----+
|       0|     1|264.0|
|       1|     1|456.0|
|       1|     0|312.0|
+--------+------+-----+
only showing top 3 rows



In [54]:
# Transformaciones con UDFs usando expresiones entre columnas
from pyspark.sql.functions import udf
titanic = load_titanic()

sex_to_num_UDF = udf(sex_to_num, IntegerType())

max_int_UDF = udf(max, IntegerType())

summary = titanic.describe().toPandas()
min_age = float(summary.loc[3,'Age'])
max_age = float(summary.loc[4,'Age'])
scale_Age_UDF = udf(lambda x : scale(x, min_age, max_age), DoubleType())
                   
titanic.select(scale_Age_UDF(titanic.Age).alias("Scaled_Age"), 
               sex_to_num_UDF(titanic.Sex).alias("Sex_Num"),
               max_int_UDF(titanic.SibSp,titanic.Parch).alias("Max_Family")).show(3)

+-------------------+-------+----------+
|         Scaled_Age|Sex_Num|Max_Family|
+-------------------+-------+----------+
| 0.2711736617240513|      1|         1|
| 0.4722292033174164|      0|         1|
|0.32143754712239253|      0|         0|
+-------------------+-------+----------+
only showing top 3 rows



In [55]:
# Una única agregación
titanic = load_titanic()

titanic.groupBy().count().show()
titanic.groupBy().sum('Survived').show()
titanic.groupBy('Pclass').sum('Survived').show()
titanic.groupBy('Pclass','Embarked').sum('Survived').show()

+-----+
|count|
+-----+
|  712|
+-----+

+-------------+
|sum(Survived)|
+-------------+
|          288|
+-------------+

+------+-------------+
|Pclass|sum(Survived)|
+------+-------------+
|     1|          120|
|     3|           85|
|     2|           83|
+------+-------------+

+------+--------+-------------+
|Pclass|Embarked|sum(Survived)|
+------+--------+-------------+
|     3|       C|           18|
|     2|       C|            8|
|     1|       Q|            1|
|     3|       Q|            6|
|     2|       Q|            1|
|     1|       C|           53|
|     1|       S|           66|
|     3|       S|           61|
|     2|       S|           74|
+------+--------+-------------+



In [56]:
# Varias agregaciones del mismo tipo
titanic = load_titanic()

titanic.groupBy('Pclass').sum('Survived', 'Fare').show()

+------+-------------+------------------+
|Pclass|sum(Survived)|         sum(Fare)|
+------+-------------+------------------+
|     1|          120| 16200.85429999999|
|     3|           85| 4696.449500000006|
|     2|           83|3714.5791999999997|
+------+-------------+------------------+



In [57]:
# Varias funciones de agregación diferentes a la vez
titanic = load_titanic()

# Usando diccionarios
titanic.groupBy('Pclass').agg({'*':'count', 'Survived':'sum'}).show()

# Usando una secuencia de funciones de la biblioteca pyspark.sql.functions
from pyspark.sql import functions
titanic.groupBy('Pclass').agg(functions.count('*').alias('Total'), functions.sum('Survived').alias('Survivors')).show()


+------+-------------+--------+
|Pclass|sum(Survived)|count(1)|
+------+-------------+--------+
|     1|          120|     184|
|     3|           85|     355|
|     2|           83|     173|
+------+-------------+--------+

+------+-----+---------+
|Pclass|Total|Survivors|
+------+-----+---------+
|     1|  184|      120|
|     3|  355|       85|
|     2|  173|       83|
+------+-----+---------+



## SQL sobre DataFrames

In [58]:
# Ejecución de código SQL que combina DataFrames
users = spark.createDataFrame([(1,'ana'),(2,'jose')],['id','nombre'])
users.createOrReplaceTempView("users")
age = spark.createDataFrame([(1,36),(2,30)],['id','edad'])
age.createOrReplaceTempView("age")

spark.sql("""SELECT *
             FROM users INNER JOIN age ON users.id == age.id""").show()

+---+------+---+----+
| id|nombre| id|edad|
+---+------+---+----+
|  1|   ana|  1|  36|
|  2|  jose|  2|  30|
+---+------+---+----+



In [59]:
titanic = load_titanic()
titanic.createOrReplaceTempView("titanic")
spark.sql("""SELECT Survived, SibSp+Parch AS Family, sex_to_num(Sex) AS Sex_Num
             FROM titanic 
             WHERE Age > 50
             """).show(3)

+--------+------+-------+
|Survived|Family|Sex_Num|
+--------+------+-------+
|       0|     0|      1|
|       1|     0|      0|
|       1|     0|      0|
+--------+------+-------+
only showing top 3 rows



In [60]:
# Ejecución de código SQL con agregación y ordenación
titanic = load_titanic()
titanic.createOrReplaceTempView("titanic")
spark.sql("""SELECT Pclass, 
                    COUNT(*) AS Total, 
                    SUM(Survived) AS Survivors 
             FROM titanic 
             GROUP BY Pclass
             ORDER BY Pclass ASC
             """).show()

+------+-----+---------+
|Pclass|Total|Survivors|
+------+-----+---------+
|     1|  184|      120|
|     2|  173|       83|
|     3|  355|       85|
+------+-----+---------+



# SparkML

## Clasificación con LinearSVC usando un Pipeline

In [61]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.classification import LinearSVC
from pyspark.ml import Pipeline

# Partimos en train (80%) + test (20%)
titanic = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
titanic = titanic.drop('PassengerId', 'Name', 'Ticket','Cabin')
titanic = titanic.dropna()

train, test = titanic.randomSplit([0.8, 0.2])

# Transforma Sex y Age a valores numéricos (orden alfabético)
indexerSex = StringIndexer(inputCol='Sex', outputCol='Sex_num', stringOrderType='alphabetAsc')
indexerEmbarked = StringIndexer(inputCol='Embarked', outputCol='Embarked_num', stringOrderType='alphabetAsc')

ohe = OneHotEncoderEstimator(inputCols=['Embarked_num'],outputCols=['Embarked_OHE'])
vec = VectorAssembler(inputCols=['Pclass', 'Sex_num', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_OHE'], outputCol='features_raw')
sca = MinMaxScaler(inputCol='features_raw', outputCol='features')
clf = LinearSVC(featuresCol='features', labelCol='Survived')

pipeline = Pipeline(stages=[indexerSex, indexerEmbarked, ohe, vec, sca, clf])


# Entrenamos el pipeline y lo usamos para clasificar 'test'
model = pipeline.fit(train)
prediction = model.transform(test)
results = prediction.select('prediction', 'Survived')
results.show(5)


# Evaluación de la clasificación usando la clase experimental MulticlassClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
claseval = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='Survived', metricName='accuracy')
print('Score:', claseval.evaluate(prediction))


# Evaluación de la regresión utilizando mllib (obsolescente)
from pyspark.mllib.evaluation import MulticlassMetrics

rdd = results.rdd.map(lambda row: (row[0], float(row[1]))) # Es necesario representar la clase Survived como float
                                                           # o MulticlassMetrics fallará
metrics = MulticlassMetrics(rdd)
print("Score:", metrics.accuracy)

ImportError: cannot import name 'OneHotEncoderEstimator' from 'pyspark.ml.feature' (/home/miguel/Documentos/PROGRAMACION/BIG DATA/env/lib/python3.10/site-packages/pyspark/ml/feature.py)

## Regresión con LinearRegression usando un Pipeline

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

# Partimos en train (80%) + test (20%)
titanic = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
titanic = titanic.drop('PassengerId', 'Name', 'Ticket','Cabin')
titanic = titanic.dropna()

train, test = titanic.randomSplit([0.8, 0.2])

# Transforma Sex y Age a valores numéricos (orden alfabético)
indexerSex = StringIndexer(inputCol='Sex', outputCol='Sex_num', stringOrderType='alphabetAsc')
indexerEmbarked = StringIndexer(inputCol='Embarked', outputCol='Embarked_num', stringOrderType='alphabetAsc')

ohe = OneHotEncoderEstimator(inputCols=['Embarked_num'],outputCols=['Embarked_OHE'])
vec = VectorAssembler(inputCols=['Pclass', 'Sex_num', 'Age', 'SibSp', 'Parch', 'Survived', 'Embarked_OHE'], outputCol='features_raw')
sca = MinMaxScaler(inputCol='features_raw', outputCol='features')
reg = LinearRegression(featuresCol='features', labelCol='Fare')

pipeline = Pipeline(stages=[indexerSex, indexerEmbarked, ohe, vec, sca, reg])


# Entrenamos el pipeline y lo usamos para inferir tarifas a partir de las instancias en 'test'
model = pipeline.fit(train)
prediction = model.transform(test)
results = prediction.select('Prediction', 'Fare')
results.show(5)

# Evaluación utilizando la clase experimental RegressionEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

maeeval = RegressionEvaluator(predictionCol='Prediction', labelCol='Fare', metricName='mae')
print("MAE :", maeeval.evaluate(results))
mseeval = RegressionEvaluator(predictionCol='Prediction', labelCol='Fare', metricName='mse')
print("MSE :", mseeval.evaluate(results))
rmseeval = RegressionEvaluator(predictionCol='Prediction', labelCol='Fare', metricName='rmse')
print("RMSE:", rmseeval.evaluate(results),'\n')

# Evaluación de la regresión utilizando mllib (obsolescente)
from pyspark.mllib.evaluation import RegressionMetrics

rdd = results.rdd
metrics = RegressionMetrics(rdd)
print("MAE :", metrics.meanAbsoluteError)
print("MSE :", metrics.meanSquaredError)
print("RMSE:", metrics.rootMeanSquaredError)

: 

## Análisis de grupos con k-means usando un Pipeline

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import ClusteringEvaluator

# Descartamos las columnas que no nos interesan y eliminamos filas con valores vacíos
titanic = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
titanic = titanic.drop('PassengerId', 'Name', 'Ticket','Cabin')
titanic = titanic.dropna()

# Transforma Sex y Age a valores numéricos (orden alfabético)
indexerSex = StringIndexer(inputCol='Sex', outputCol='Sex_num', stringOrderType='alphabetAsc')
indexerEmbarked = StringIndexer(inputCol='Embarked', outputCol='Embarked_num', stringOrderType='alphabetAsc')
# One-hot-encoding de la columna Embarked
ohe = OneHotEncoderEstimator(inputCols=['Embarked_num'],outputCols=['Embarked_OHE'])
# Combina todas las columnas en un único vector
vec = VectorAssembler(inputCols=['Pclass', 'Sex_num', 'Age', 'SibSp', 'Parch', 'Survived', 'Embarked_OHE','Fare'], outputCol='features_raw')
# Normaliza los valores de cada posición del vector
sca = MinMaxScaler(inputCol='features_raw', outputCol='features')
# Realiza análisis de grupos
clu = KMeans(k=3) # valores por defecto: atributos en 'features' y centroide en 'prediction'


pipeline = Pipeline(stages=[indexerSex, indexerEmbarked, ohe, vec, sca, clu])


# Entrenamos el pipeline
model = pipeline.fit(titanic)
prediction = model.transform(titanic)
prediction.select('prediction').show(5)

# Obtenemos los centroides desde el último modelo del pipiline
print('Centroides:')
print(type(model.stages[-1].clusterCenters()[0]))
for c in model.stages[-1].clusterCenters():
    print(c)

# Evaluación utilizando la clase experimental ClusteringEvaluator para obtener el coeficiente de silueta
evaluator = ClusteringEvaluator()
print('Silhouette Coefficient:', evaluator.evaluate(prediction))

# No existen métodos de evaluación de clústers en mllib

: 

## Persistencia de modelos en Pipelines

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

# Partimos en train (80%) + test (20%)
titanic = spark.read.csv('../../data/Cap7/titanic.csv', header=True, inferSchema=True)
titanic = titanic.drop('PassengerId', 'Name', 'Ticket','Cabin')
titanic = titanic.dropna()

train, test = titanic.randomSplit([0.8, 0.2])

# Transforma Sex y Age a valores numéricos (orden alfabético)
indexerSex = StringIndexer(inputCol='Sex', outputCol='Sex_num', stringOrderType='alphabetAsc')
indexerEmbarked = StringIndexer(inputCol='Embarked', outputCol='Embarked_num', stringOrderType='alphabetAsc')

ohe = OneHotEncoderEstimator(inputCols=['Embarked_num'],outputCols=['Embarked_OHE'])
vec = VectorAssembler(inputCols=['Pclass', 'Sex_num', 'Age', 'SibSp', 'Parch', 'Survived', 'Embarked_OHE'], outputCol='features_raw')
sca = MinMaxScaler(inputCol='features_raw', outputCol='features')
reg = LinearRegression(featuresCol='features', labelCol='Fare')

pipeline = Pipeline(stages=[indexerSex, indexerEmbarked, ohe, vec, sca, reg])

# Entrenamos el pipeline y lo usamos para inferir tarifas a partir de las instancias en 'test'
model = pipeline.fit(train)
prediction = model.transform(test)
results = prediction.select('Prediction', 'Fare')
results.show(5)

# Volcado del modelo
model.save('../../data/Cap7/regression_model') # Equivalente a write().save(path)
#model.write().overwrite().save('../../data/Cap7/regression_model')

: 

In [ ]:
from pyspark.ml import PipelineModel

# Carga del modelo
loaded_model = PipelineModel.load('../../data/Cap7/regression_model')

prediction = loaded_model.transform(test)
results = prediction.select('Prediction', 'Fare')
results.show(5)

: 